In [1]:
# import dependencies 
import pandas as pd 
import matplotlib.pyplot as lib 
import numpy as np

# Import the API key.
from config import weather_api_key

import time
from datetime import datetime 
from citipy import citipy

# Import the requests library.
import requests




In [2]:
# store the lat and long
# delcare each array as var 
# 6.1.4

# generate random lat 
lats = np.random.uniform(low=-90.000, high = 90.000, size = 1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size = 1500)

# create zip object in memory (packing the two together)
lat_lngs = zip(lats, lngs)
# packed into tuple (1500 of them) and access as 0,1 index form
lat_lngs


In [3]:
# unpack the lat_lngs zip object into list type
# cannot simply print out coordinates on its own without using 0,1 indexing
coordinates = list(lat_lngs)

In [4]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-87.00892355356804 -103.36779367508521
3.278751783930659 -27.0909963096056
52.980145123356436 33.45540591166298
-31.61696317742213 131.10165998288045
45.11384721771603 -111.23881797336365
85.98977242388253 162.3973738632418
51.386430561457786 -79.54326302613251
-29.10700752841155 83.0422355213804
62.39347322049673 76.15918671648575
13.757054477056897 104.76694022420406
-82.14961543551658 -158.2026658735945
-48.91180087993613 -85.40268407606897
31.3817273224753 -105.24808391726171
-37.061789953809715 49.68118792602155
-83.00465457189662 95.26216248042306
-63.3352553257415 -90.24394563187508
25.233936461779777 -133.44879893857905
-72.94754263429874 -160.2646295389846
40.3746580387739 6.66114324187987
-27.056505935009476 5.991856911615713
-21.99103702960315 -113.20322384491853
-70.16951304280397 -111.20711125509246
-24.745836982129816 61.75266648582348
0.4055107115421066 31.688609427875605
14.53442519993905 -133.40256423056664
-11.399795294180308 -21.15130355121667
-14.941205339877243 164

In [5]:
# 6.1.5 generate random wolrd cities 
# Use the citipy module to determine city based on latitude and longitude.
# iterate through the coordinates unzipped tuple 
# use citypy and add lat, long
# print the city name, chain the name to the nearest city function
# print the country name, chain the country code to nearest city function 

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
#print(cities)
len(cities)

621

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=fd1fbd1a397ad51b5a5aa7507997d4cf


In [ ]:
# 1. import dependencies, initialize counters and a list to hold weather data
city_data = []
print("Beginning Data Retrieval           ")
print("-----------------------------------")

# create counters
# initialize to 1 b/c we want first iteration of logging for each response 
record_count = 1
set_count = 1

# fixed version that adds not city index, but name 
# enumerate helps refrence the index and city in list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    # remove blank spaces in city name with .replace
    city_url = url + "&q=" + city.replace(" ","+")

    # create print statement to tell us record and set counts 
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        #print(city_weather)
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# 6.2.7 create the df for city weather data
# now we have a list of dict, use pandas to make our df
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
print(city_data)
#city_data_df.head(10)

In [ ]:
# 6.3.1 plot lat vs temp
# Extract relevant fields from the DataFrame for plotting.
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]

In [ ]:

# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,
            max_temps,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
#plt.savefig("weather_data/Fig1.png")

# Show plot.
plt.show()

In [ ]:
# 6.3.2 plot lat vs humidity
# Build the scatter plots for latitude vs. humidity.
plt.scatter(lats,
            humidity,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Humidity "+ time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig2.png")
# Show plot.
plt.show()